In [1]:
import os

import torch
import wandb
from hydra import compose, initialize
from hydra.utils import instantiate
from metalearning_benchmarks import MetaLearningBenchmark
from omegaconf import OmegaConf
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

from src.components.contextual_dvi import ContextualDVI
from src.components.control import Control
from src.components.decoder import Decoder
from src.components.dvi_process import DiffusionVIProcess
from src.components.encoder import SetEncoder
from src.components.hyper_net import HyperNet
from src.utils.context_datasets import MetaLearningDataset
from src.utils.train import train
from src.utils.visualize import visualize

In [2]:
try:
    import torch_directml
    device = torch_directml.device() 
except ImportError:
    device = torch.device("cpu") 
print(device)

privateuseone:0


In [3]:
dir = "../models/revived-plasma-50" # "../models/test" # 

In [4]:
with initialize(version_base=None, config_path=dir):
    config = compose(config_name="config")

    benchmark: MetaLearningBenchmark = instantiate(config.benchmark.value)
    dataset = MetaLearningDataset(benchmark=benchmark)
    dataloader = DataLoader(dataset=dataset, batch_size=config.training.value.batch_size, shuffle=True)

    set_encoder = SetEncoder(
        c_dim=config.common.value.c_dim,
        h_dim=config.common.value.h_dim,
        num_layers=config.common.value.num_layers,
        non_linearity=config.common.value.non_linearity,
        is_attentive=config.set_encoder.value.is_attentive,
        is_aggregative=not config.control_and_hyper_net.value.is_cross_attentive
        or not config.decoder.value.is_cross_attentive,
        is_non_aggregative=config.control_and_hyper_net.value.is_cross_attentive
        or config.decoder.value.is_cross_attentive,
        use_context_size=config.set_encoder.value.use_context_size,
        aggregation=config.set_encoder.value.aggregation,
        max_context_size=dataset.max_context_size,
    )

    control = Control(
        h_dim=config.common.value.h_dim,
        z_dim=config.common.value.z_dim,
        num_layers=config.common.value.num_layers,
        non_linearity=config.common.value.non_linearity,
        num_steps=config.dvi_process.value.num_steps,
        is_cross_attentive=config.control_and_hyper_net.value.is_cross_attentive,
        num_heads=config.control_and_hyper_net.value.num_heads,
    )

    hyper_net = (
        HyperNet(
            h_dim=config.common.value.h_dim,
            z_dim=config.common.value.z_dim,
            non_linearity=config.common.value.non_linearity,
            num_steps=config.dvi_process.value.num_steps,
            is_cross_attentive=config.control_and_hyper_net.value.is_cross_attentive,
            num_heads=config.control_and_hyper_net.value.num_heads,
        )
        if config.control_and_hyper_net.value.use_hyper_net
        else None
    )

    dvi_process: DiffusionVIProcess = instantiate(
        config.dvi_process.value,
        z_dim=config.common.value.z_dim,
        control=control,
        hyper_net=hyper_net,
        device=device,
    )

    decoder = Decoder(
        x_dim=config.common.value.x_dim,
        z_dim=config.common.value.z_dim,
        h_dim=config.common.value.h_dim,
        y_dim=config.common.value.y_dim,
        num_layers=config.common.value.num_layers,
        non_linearity=config.common.value.non_linearity,
        has_lat_path=config.decoder.value.has_lat_path,
        has_det_path=config.decoder.value.has_det_path,
        is_cross_attentive=config.decoder.value.is_cross_attentive,
        num_heads=config.decoder.value.num_heads,
    )

    contextual_dvi = ContextualDVI(
        encoder=set_encoder,
        dvi_process=dvi_process,
        decoder=decoder,
    ).to(device)

    path = f"{dir}/contextual_dvi_finetuned.pth"
    if os.path.exists(path):
        state_dict = torch.load(path, map_location=torch.device("cpu"), weights_only=False)
        contextual_dvi.load_state_dict(state_dict)
        print(f"Loaded model from {path}")

Generating tasks: 100%|██████████| 4096/4096 [00:00<00:00, 282959.22it/s]

Loaded model from ../models/revived-plasma-50/contextual_dvi_finetuned.pth


In [5]:
wandb_logging = False

if wandb_logging:
    wandb.init(project="dvi-bml-test")

In [8]:
num_epochs = 1000
# params = [
#     {"params": dvi_process.parameters(), "lr": config.learning_rate},
#     {"params": set_encoder.parameters(), "lr": config.learning_rate},
#     {"params": decoder.parameters(), "lr": config.learning_rate}
# ]
optimizer = torch.optim.AdamW(contextual_dvi.parameters(), lr=0.1 *config.training.value.learning_rate)
scheduler = None #ReduceLROnPlateau(optimizer, mode="min", factor=0.3, patience=500)

In [9]:
losses = train(
    device=device, 
    contextual_dvi=contextual_dvi,
    target_constructor=None,
    num_epochs=num_epochs,
    dataloader=dataloader, 
    optimizer=optimizer, 
    scheduler=scheduler,
    max_clip_norm=None, #0.1, # config.training.value.max_clip_norm,
    wandb_logging=wandb_logging,
    alpha=1.0
)

 25%|██▌       | 1/4 [00:00<00:02,  1.13it/s, epoch=8, loss=251, lr=None]


KeyboardInterrupt: 

In [11]:
gen_dataloader = DataLoader(dataset, 1, True)

max_context_size = 6
num_samples = 1024

visualize(
    device=device,
    dvi_process=dvi_process,
    set_encoder=set_encoder,
    decoder=decoder,
    dataloader=gen_dataloader,
    control=control,
    config=config,
    num_samples=num_samples,
    max_context_size=max_context_size
)

In [25]:
torch.save(contextual_dvi.state_dict(), f"{dir}/contextual_dvi_finetuned.pth")